<a href="https://colab.research.google.com/github/adalves-ufabc/2021.QS-PLN/blob/main/2021_Q1_PLN_Notebook_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2021.Q1]**

Prof. Alexandre Donizeti Alves

##**Modelo de Linguagem com N-gramas**


### **Bibliotecas**


In [1]:
# expressoes regulares
import re

# cria um dicinario com a frequencia dos termos em um iterable
from collections import Counter

## subsequencias de um iterable
from itertools import islice

# numeros e sequencias aleatoreas
import random

### **Funções principais**

In [2]:
regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+"       # raw string
#regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ0-9]+"   # raw string

# usa expressoes regulares para quebrar um texto em tokens
def get_tokens(fileName):
  # leitura do documento
  with open(fileName,'r') as document:
     content  = document.read()  # devolve um vetor contendo as linhas do arquivo
     content  = content.lower()

  Words    = re.findall(regex, content)

  return(Words)

# similar ao get_tokens, mas removindo elementos da lista de stopwords
def get_tokens_without_stopwords(fileName,stopwordsName="/content/stopwords.txt"):
   # leitura do documento
   with open(fileName,'r') as document:
      content  = document.read()  # devolve um vetor contendo as linhas do arquivo
      content  = content.lower()

   # leitura das stopwords
   with open(stopwordsName,'r') as stopwordsfile:
      stopwords = set([])
      for s in stopwordsfile.readlines():                                                                                                                                                     
        stopwords.add(s.strip().lower())

   # remove as stopwords
   Words    = [w for w in re.findall(regex, content) if w not in stopwords]

   return(Words)

# retirado de um exemplo na internet
def window(seq, n=2):
    "Retorna uma janela deslizante (de tamanho n) sobre a sequencia seq"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

# isa o Counter para contar a frequencia de unigramas e bigramas
def ngrams(Words):
    "Retarna a contagem de unigramas e bigramas a partir da lista de palavras"
  
    # Conta Unigramas (utilizando o counter de collections)
    Unigrams = Counter(Words)

    # windows retora uma janela delizante de tamanho 2
    Bigrams  = Counter(window(Words,2))

    return(Unigrams,Bigrams)

# funcao auxiliar para calcular as probabilidades
BigramProbabilities = lambda w1, w2 : bigrams [ (w1,w2) ] / unigrams[ w1 ]

# aplica o score em uma lista de sentencas
def score(phrases):

    # loop sobre todas as sentencas de teste
    for phrase in phrases:
        Words = re.findall(regex, phrase)
        P = float(1.0)
        for w_0, w_1 in window(Words,2):
            P = P * BigramProbabilities(w_0, w_1)

        print ( "{1:.20f} : Sentença: {0}".format( phrase, P ) ) 

### **Testando o Modelo**

Cria uma sequência de sentenças e um modelo de bigramas baseado no livro de **Machado de Assis**, e aplica a função score na lista de sentenças

In [3]:
sentencas = ["ele é",
             "ele é uma", 
             "ele é uma pessoa", 
             "ele é uma pessoa de", 
             "ele é uma pessoa de verdade"]

words = get_tokens("/content/A-Semana-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)
score(sentencas)

0.02647657841140529586 : Sentença: ele é
0.00069189664838167149 : Sentença: ele é uma
0.00000842133213707000 : Sentença: ele é uma pessoa
0.00000050654629395910 : Sentença: ele é uma pessoa de
0.00000000051863038186 : Sentença: ele é uma pessoa de verdade


Repetimos o processo, agora considerando todas as obras de Machado de Assis

In [4]:
sentencas = ["ele é",
             "ele é uma", 
             "ele é uma pessoa", 
             "ele é uma pessoa de", 
             "ele é uma pessoa de verdade"]

words = get_tokens("/content/Todas-as-obras-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)
score(sentencas)

0.02488038277511961729 : Sentença: ele é
0.00093406050182903407 : Sentença: ele é uma
0.00000825483405278844 : Sentença: ele é uma pessoa
0.00000046590235609081 : Sentença: ele é uma pessoa de
0.00000000050723310908 : Sentença: ele é uma pessoa de verdade


In [ ]:
print(len(words))

1535663


In [ ]:
print(len(unigrams))

63328


In [ ]:
print(len(bigrams))

618262


### **Probabilidade da próxima palavra**

Usamos o modelo de bigrama para calcular quais as palavras mais prováveis.

In [5]:
def next_prob(phrase,n=5):
    # quebre a sentenca em palavras
    Words = re.findall(regex, phrase)

    # calcula as probabilidades de todas as palavras em que o bigrama eh w1 e armazena em prob
    probs = {w2 : BigramProbabilities(w1,w2) for (w1,w2) in bigrams.keys() if w1 == Words[-1] }

    # ordena e imprime as n mais relevantes
    for w, p in islice(sorted(probs.items(), key = lambda item: item[1], reverse=True),n):
        print ( "{0} -> {1} ({2:.2f}%)".format( phrase, w.upper(), p*100 ) )   

**Teste da função `next_prob`**

O usuário digita frases e modelo lista as 5 palavras mais prováveis de acordo com o modelo.

In [8]:
words = get_tokens("/content/A-Semana-Machado-de-Assis.txt")
#words = get_tokens("/content/Todas-as-obras-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)

phrase = input("\nDigite uma frase: ")

while (phrase != ""):
    next_prob(phrase)
    phrase = input("\nDigite uma frase:")

# frase: ele é uma pessoa
# frase: estudar
# frase: a semana que


Digite uma frase: ele é uma pessoa
ele é uma pessoa -> QUE (28.57%)
ele é uma pessoa -> DE (6.02%)
ele é uma pessoa -> A (4.51%)
ele é uma pessoa -> É (2.26%)
ele é uma pessoa -> NO (2.26%)

Digite uma frase:


**Teste da função `next_prob`**

O usuário digita frases e modelo lista as 5 palavras mais prováveis de acordo com o modelo, **sem considerar as *stopwords***

In [9]:
words = get_tokens_without_stopwords("/content/A-Semana-Machado-de-Assis.txt")

unigrams, bigrams = ngrams(words)

phrase = input("\nDigite uma frase: ")

while (phrase != ""):
    next_prob(phrase)
    phrase = input("\nDigite uma frase:")

# frase: que
# frase: rio


Digite uma frase: que

Digite uma frase:a

Digite uma frase:Rio

Digite uma frase:rio
rio -> JANEIRO (54.70%)
rio -> GRANDE (15.38%)
rio -> BRANCO (5.98%)
rio -> CLARO (2.56%)
rio -> NEWS (1.71%)

Digite uma frase:


### **Classe NGrams**

- Reorganiza e empacota as funções anteriores em uma classe

- O parâmetro 'max_n' corresponde ao valor máximo de elementos no modelo *ngram*

- Uma única tabela `Counter` é usada, pois a própria chave já contém a informação do número de *grams*

- O método `probability` trata n-gramas que não aparecem na base

- O método `generate` pega a primeira palavra acima de um `threshold` em uma lista de unigramas embaralhada

In [13]:
class NGrams(object):

    def __init__(self,max_n,Words=None):
        self.max_n   = max_n
        self.Counts  = Counter()

        if Words is not None:
            self.update(Words)

    def update(self, Words):

        # conta os unigram, bigram, trigram, ..., ngram
        # e armazena na mesma estrutura
        for i in range(1,self.max_n+1):
            self.Counts.update(window(Words,i))

        # Caso especial: tupla vazia (util para o metodo 'probability')
        # O valor eh igual ao numero de palavras
        self.Counts[()] += len(Words)

    # Calcula a probabilidade para a frase: Words
    def probability(self, Words):
        if len(Words) <= self.max_n:
            return self._probability(Words)
        else:
            P = 1
            for i in range(len(Words) - self.max_n + 1):
                ngram = Words[i:i + self.max_n]
                P     = P * self._probability(ngram)
            return P

    # Calcula a aproximacao para o n-grama usando seu prefixo
    def _probability(self, ngram):
        ngram        = tuple(ngram)
        ngram_count  = self.Counts[ngram]
        prefix_count = self.Counts[ngram[:-1]]

        # Se uma tupla (n-grama) nao for observada devolvemos zero
        if ngram_count and prefix_count:
            return ngram_count / prefix_count
        else:
            return 0.0

        # Geracao de frases de 'n_words'
    def generate(self, n_words, threshold = random.random()):

        # cria uma lista de unigrams
        unigrams = [ngram for ngram in self.Counts.keys() if len(ngram) == 1]

        # Tentamos gerar frases 
        words = []
        
        while len(words) < n_words:
            # o prefixo para o proximo n-grama
            if self.max_n == 1:
                prefix = ()
            else:
                prefix = tuple(words[-self.max_n + 1:])
           
            total     = 0.0
            random.shuffle(unigrams)
            for unigram in unigrams:
                total += self._probability(prefix + unigram)
                if total >= threshold:
                    words.extend(unigram)
                    break
            
            # Se nao for possivel criar uma frase  
            if total == 0.0:
                raise RuntimeError('impossible sequence')

        return(words)


### **Teste geração de sentenças**

Usamos os n-gramas de todas as obras de Machado de Assis com 1, 2 e 5 gramas



In [14]:
words = get_tokens("/content/Todas-as-obras-Machado-de-Assis.txt")

In [16]:
# unigrama
ng = NGrams(1, Words=words)
print(" ".join(ng.generate(30)))

em das espécie historias amedrontar não jesuine da em a é foi não mãos a nomes prole voltaram da do dirão algumas dali do os dos mesma de em de


In [19]:
# bigrama
ng = NGrams(2, Words=words)
print(" ".join(ng.generate(30)))

lhe disse que de desdém forcejando por que sou poeta a um deles não mas como me indigna margarida mas essa luta que além disso é na casaca -------------------------------------------------------------------------- texto-fonte


In [20]:
# 5-grama
ng = NGrams(5, Words=words)
print(" ".join(ng.generate(30)))

comia tão pouco que quase não olhava para o diplomata nem para o chão nem para as paredes ou móveis mas para o ar como aires insistisse ele acordou e


Repetimos o teste, agora usando os n-gramas do **discurso do Bolsonaro na ONU**. Observem a dependência do corpus usado


In [21]:
words = get_tokens("/content/discurso_bolsonaro.txt")

In [22]:
ng = NGrams(3, Words=words)
print(" ".join(ng.generate(30)))

mais hospitaleiro acabamos de estender a isenção de vistos para países como estados unidos japão austrália e canadá e estamos estudando adotar medidas similares para china e índia dentre outros


### **Geração de sentenças com pontuação**

Considera os caracteres de pontuação e números como sendo gramas válidos

In [23]:
regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+|[,\.-?]|[0-9]+"   # raw string

words = get_tokens("/content/Todas-as-obras-Machado-de-Assis.txt")

ng = NGrams(5, Words=words)
print(" ".join(ng.generate(30)))

das noites belas , o terrível cenário preparava a mais terríveis lances . então surge dos tronos a profética voz que anunciava ao teu crédulo esposo : tu serás rei


In [24]:
regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+|[,\.-?]|[0-9]+"   # raw string

words = get_tokens("/content/discurso_bolsonaro.txt")

ng = NGrams(3,Words=words)
print(" ".join(ng.generate(50)))

disfarçados de boas intenções . estamos prontos para , em busca de prosperidade , estamos adotando políticas que nos é mais sagrado : a nossa soberania um deles por ocasião do encontro do g 7 ousou sugerir aplicar sanções ao brasil , sem sequer nos ouvir . agradeço a deus
